# Example Brick Model for process
Example brick model will be 2 AHUs, 60 terminal units.
Half are for perimeter zones vavs with reheat, half are for core zones and are cooling only 

10 core zone terminal units are for mechanical rooms, networking closets, and electrical rooms that must be served 24/7

In [58]:
from rdflib import Namespace, URIRef, RDFS, Graph, Literal
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library
from buildingmotif.namespaces import BMOTIF, BRICK, A, RDF
from buildingmotif.model_builder import TemplateBuilderContext

In [2]:
BLDG = Namespace("urn:example#")
bm = BuildingMOTIF("sqlite://", "topquadrant")
bldg = Model.create("urn:example#")
bldg.graph.bind('',BLDG)
brick = Library.load(ontology_graph="Brick.ttl")

2025-07-06 13:34:43,295 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/sou (Name: http://qudt.org/2.1/vocab/sou). Check this is loaded into BuildingMOTIF
2025-07-06 13:34:43,296 | root |  WARNING: An ontology could not resolve a dependency on https://w3id.org/rec (Name: https://w3id.org/rec). Check this is loaded into BuildingMOTIF
2025-07-06 13:34:43,296 | root |  WARNING: An ontology could not resolve a dependency on https://w3id.org/rec/recimports (Name: https://w3id.org/rec/recimports). Check this is loaded into BuildingMOTIF
2025-07-06 13:34:43,297 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/schema/shacl/overlay/qudt (Name: http://qudt.org/2.1/schema/shacl/overlay/qudt). Check this is loaded into BuildingMOTIF
2025-07-06 13:34:43,297 | root |  WARNING: An ontology could not resolve a dependency on http://qudt.org/2.1/vocab/prefix (Name: http://qudt.org/2.1/vocab/prefix). Check this is loaded into 

In [3]:
brick_tmpl = Library.load(directory='brick-templates')

In [4]:
# start with the tank. This instantiates the template with 'my_tank' as the name
# You can assign other parameters here as well. If the value you give to a parameter
# is a string (e.g. 'my_tank') then the builder will automatically make a URL with the BLDG (see cell above) as the namespace
# ahu = ctx['multiple-zone-ahu'](name='ahu1')
# # now create an instance of the water flow property
# flow_prop = ctx['water-flow'](name='tank_outlet_water_flow')
# flow_prop['name-owner'] = my_tank['out']
# # now create the sensor
# flow_sensor = ctx['sensor'](name='my_flow_sensor')
# flow_sensor['where'] = my_tank['out'] # the sensor measures at the outlet of the tank
# flow_sensor['property'] = flow_prop['name'] # associate the sensor's "property" param with the flow property

In [5]:
# bldg_config
# assuming cooling+only and reheat vavs split evenly. Computer spaces are all in the cooling only vavs.
# [ahus, cooling_only_vavs_per_ahu, reheat_vavs_per_ahu]
# Could add amt computer spaces_per_ahu to building if it serves example later

# the context helps us build up our model
ctx = TemplateBuilderContext(BLDG)
ctx.add_templates_from_library(brick)
ctx.add_templates_from_library(brick_tmpl)

In [6]:
# preparing both representations 

In [7]:
# bldg_config = {'ahus': 2, 'cv': 10, 'hv': 10}
# bldg_ctx_dict = {}
# for i in range(bldg_config['ahus']):
#     ahu_name = f'ahu_{i}'
#     # bldg_ctx_dict[name] = ctx['multiple-zone-ahu'](name=name)
#     ahu = ctx['multiple-zone-ahu'](name=ahu_name)
#     for cv in range(bldg_config['cv']):
#         cv_name = f'cvav_{i}_{cv}'
#         cvav = ctx['vav-cooling-only'](name=cv_name)
#         ctx['feeds'](name = ahu['name'], target = cvav['name'])
#     for hv in range(bldg_config['hv']):
#         hv_name = f'hvav_{i}_{hv}'
#         hvav = ctx['vav-with-reheat'](name=hv_name)
#         ctx['feeds'](name = ahu['name'], target = hvav['name'])

In [8]:
# TODO: MESSED UP CODE FLOW, NOW NEED TO RUN AGAIN FROM HERE
def get_wrapper_dict(ctx):
    wrapper_dict = {}
    for wrapper in ctx.wrappers:
        if wrapper.template.name not in wrapper_dict.keys():
            wrapper_dict[wrapper.template.name] = [wrapper]
        else:
            wrapper_dict[wrapper.template.name].append(wrapper)
    return wrapper_dict

In [9]:
wrapper_dict = get_wrapper_dict(ctx)
template_types = wrapper_dict.keys()
print(template_types)

dict_keys([])


In [10]:
TEMPLATE_PARAMS_COMPILED = {
    "vav-cooling-only": {
        "name": "VAV",
        "ztemp": "Zone_Air_Temperature_Sensor",
        "occ": "Occupancy_Sensor",           # optional
        "co2": "CO2_Level_Sensor",           # optional
        "dat": "Discharge_Air_Temperature_Sensor",
        "dmp": "Damper",
        "dmp-dmppos": "Damper_Position_Command",
        "zone": "HVAC_Zone"
    },
    "vav-with-reheat": {
        "name": "VAV",
        "name-ztemp": "Zone_Air_Temperature_Sensor",
        "name-occ": "Occupancy_Sensor",           # optional
        "name-co2": "CO2_Level_Sensor",           # optional
        "name-dat": "Discharge_Air_Temperature_Sensor",
        "name-dmp": "Damper",
        "name-dmp-dmppos": "Damper_Position_Command",
        "name-zone": "HVAC_Zone",
        "rhc": "Heating_Coil",
        "rhc-cmd": "Position_Command"
    },
    "multiple-zone-ahu": {
        "name": "AHU",
        "sa_fan": "Supply_Fan",
        "sa_fan-start_stop": "Start_Stop_Command",
        "sa_fan-speed": "Frequency_Command",
        "sa_fan-status": "Fan_Status",       # optional
        "ra_damper": "Damper",
        "ra_damper-dmppos": "Damper_Position_Command",
        "clg_coil": "Cooling_Coil",
        "clg_coil-valve_cmd": "Valve_Position_Command",
        "htg_coil": "Heating_Coil",
        "htg_coil-valve_cmd": "Valve_Position_Command",
        "sa_temp": "Supply_Air_Temperature_Sensor",
        "oa_temp": "Outside_Air_Temperature_Sensor",
        "ma_temp": "Mixed_Air_Temperature_Sensor",    # optional
        "ra_temp": "Return_Air_Temperature_Sensor",   # optional
        "filter_pd": "Filter_Differential_Pressure_Sensor"  # optional
    },
    "with-relief-damper": {
        "name": "AHU",   # composed on the AHU's 'name'
        "relief-fan": "Exhaust_Fan",
        "relief-damper": "Damper",
        "relief-damper-dmppos": "Damper_Position_Command",
        "relief-damper": "Exhaust_Damper",
        "sp-sensor": "Static_Pressure_Sensor"   # optional
    },
    "with-return-fan": {
        "name": "AHU",   # composed on the AHU's 'name'
        "return-fan": "Return_Fan",
        "relief-damper": "Damper",
        "relief-damper-dmppos": "Damper_Position_Command",
        "relief-damper": "Exhaust_Damper",
        "sp-sensor": "Static_Pressure_Sensor",          # optional
        "supply-air-flow": "Supply_Air_Flow_Sensor",    # optional
        "return-air-flow": "Return_Air_Flow_Sensor"     # optional
    },
    "relief-fan": {
        "name": "Exhaust_Fan"
    },
    "return-fan": {
        "name": "Return_Fan"
    },
    "fan": {
        "name": "Fan",
        "speed": "Frequency_Command",
        "start_stop": "Start_Stop_Command",
        "status": "Fan_Status"
    },
    "damper": {
        "name": "Damper",
        "dmppos": "Damper_Position_Command"
    },
    "heating-coil": {
        "name": "Heating_Coil",
        "cmd": "Position_Command"
    },
    "sa_fan": {
        "name": "Supply_Fan",
        "start_stop": "Start_Stop_Command",
        "speed": "Frequency_Command",
        "status": "Fan_Status"   # optional
    },
    "clg_coil": {
        "name": "Cooling_Coil",
        "valve_cmd": "Valve_Position_Command"
    },
    "htg_coil": {
        "name": "Heating_Coil",
        "valve_cmd": "Valve_Position_Command"
    },
    "feeds": {
        "name": None,    # The subject of the feeds relation (type depends on context)
        "target": None   # The object of the feeds relation (type depends on context)
    }
}

In [ ]:
g = Graph()
g.bind('',BLDG)

bldg_config = {'ahus': 2, 'cv': 30, 'hv': 30}
bldg_ctx_dict = {}
for i in range(bldg_config['ahus']):
    template_name = 'multiple-zone-ahu'
    ahu_name = f'ahu_{i}'
    ahu = ctx[template_name](name=ahu_name)
    ahu.bindings = {k: BLDG[f"{template_name}_{k}_{i}"] for k in ahu.parameters}
    # display(ahu.bindings)
    # also adding to condensed representation 
    for k,v in ahu.bindings.items():
        g.add((BLDG[f"{template_name}-{k.replace('name-','')}"], RDF[f"_{i+1}"], v))
    for cv in range(bldg_config['cv']):
        template_name = 'vav-cooling-only'
        cv_name = f'cvav_{i}_{cv}'
        cvav = ctx[template_name](name=cv_name)
        cvav.bindings = {k: BLDG[f"{template_name}_{k}_{i}_{cv}"] for k in cvav.parameters}
        ctx['feeds'](name = ahu['name'], target = cvav['name'])
        for k,v in cvav.bindings.items():
            g.add((BLDG[f"{template_name}-{k.replace('name-','')}"], RDF[f"_{cv+1}"], v))
        # print(cvav.bindings)
    for hv in range(bldg_config['hv']):
        template_name = 'vav-with-reheat'
        hv_name = f'hvav_{i}_{hv}'
        hvav = ctx[template_name](name=hv_name)
        hvav.bindings = {k: BLDG[f"{template_name}_{k}_{i}_{hv}"] for k in hvav.parameters}
        ctx['feeds'](name = ahu['name'], target = hvav['name'])
        for k,v in hvav.bindings.items():
            g.add((BLDG[f"{template_name}-{k.replace('name-','')}"], RDF[f"_{hv+1}"], v))

wrapper_dict = get_wrapper_dict(ctx)
template_types = wrapper_dict.keys()

skip_templates = ['feeds']
for tp in template_types:
    if tp in skip_templates:
        continue
    for wrapper in wrapper_dict[tp]:
        params = wrapper.parameters
        eval_dict = {}
        for param in params:
            # if param not in TEMPLATE_PARAMS_COMPILED[tp].keys():
            #     print(param)
            # g.add((BLDG[f"{tp}-{param.replace('name-','')}"], A, BRICK[TEMPLATE_PARAMS_COMPILED[tp][param]]))
            name = f"{tp}-{param.replace('name-','')}"
            g.add((BLDG[name], A, RDF.Seq))
            eval_dict[param] = BLDG[name]
        template_graph = wrapper.template.evaluate(eval_dict)
        if not isinstance(template_graph, Graph):
            print('not a graph, cant add')
        g = g + template_graph
    # adding feeds manually 
    g.add((BLDG['multiple-zone-ahu-name'], BRICK.feeds, BLDG['vav-cooling-only-name']))
    g.add((BLDG['multiple-zone-ahu-name'], BRICK.feeds, BLDG['vav-with-reheat-name']))

In [21]:
g.serialize('brick-example-groups.ttl', format = 'ttl')

<Graph identifier=Ne87640ca464a47dab6d949aa0bb82e1f (<class 'rdflib.graph.Graph'>)>

In [77]:
g_no_data = g.query("""
            CONSTRUCT {
                   ?s ?p ?o .
                    ?s a ?type .
                    ?o a ?otype .
            }
            WHERE {
            ?s ?p ?o .
            ?s a ?type .
            ?o a ?otype .
            FILTER EXISTS {
                    ?o a rdf:Seq .
                }
            FILTER (?type != rdf:Seq) .
            
            FILTER (?otype != rdf:Seq) .
            }
                   """).graph

def get_label(s):
    name= str(s).rsplit('-')[-1] if str(s).rsplit('-')[-1] != 'name' else str(s).rsplit('-')[-2]
    return Literal(name)
for s,p,o in g_no_data.triples((None,None,None)):    
    if p == A:
        continue               
    if p == BRICK.hasPoint:
        g_no_data.add((o, A, BRICK.Point))
    g_no_data.add((s, RDFS.label, get_label(s)))
    g_no_data.add((o, RDFS.label, get_label(o)))

g_no_data.bind('bldg',BLDG)

In [78]:
g_no_data.serialize("brick-example-groups-no-data.ttl", format = 'ttl')

<Graph identifier=Nedc38380520a4e23ae06241eb7f0b9ae (<class 'rdflib.graph.Graph'>)>

In [53]:
bldg.add_graph(ctx.compile())
bldg.graph.serialize('brick-example.ttl', format = 'ttl')

<Graph identifier=fd453114-7cdb-4326-b3a6-5a3d472837e3 (<class 'rdflib.graph.Graph'>)>